<a href="https://colab.research.google.com/github/Paloma-Moya-Tudela/AA_PRACTICA1_GRUPO_2/blob/main/A_PRACTICA2_GRUPO_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**GRUPO 2**

María García Rodríguez

Paloma Moya Tudela

Sofía Peña González

Elena Peral Pastor



# RESUMEN

# ÍNDICE



*   Introducción
*   Resolución
*   Conclusiones
*   Reflexiones
*   Referencias Bibliográficas







# INTRODUCCIÓN

# RESOLUCIÓN

El primer paso será comprender el enunciado y extraer los datos que se nos aportan.

**PROBLEMA:**

*

Una vez se tienen claro los materiales con los que se trabajará, se podrá empezar a incorporar código. Sin embargo, antes de comenzar importemnos keras y el resto de librerías necesarias para la red neuronal, asegurando que es la versión 2.0.0 (o superior) de TensorFlow.

## Configuración de los casos

***1. Configurar y entrenar los siguientes modelos de red de neuronas convolucionales, analizando y reflexionando sobre los resultados.***

**Caso 1**

**Caso 2**

**Caso 3**

## Salidas

***2. Explicar la salida de la llamada model.summary() de cada uno de los 3 casos.***

##Análisis (5 épocas vs 25 épocas)

***3. Analizar e interpretar los resultados del Caso 1 frente a su original si se multiplica por 5 las épocas de entrenamiento (25).***

##Análisis (5 épocas vs 25 épocas)

***4. Analiza el resultado del Caso 1 si en lugar de 'ReLU' usas 'tanh' en la función de activación de las dos capas convolucionales..***

## Comparación de rendimiento

***5. Evaluar cada uno de los 3 modelos comparando el rendimiento del modelo en el conjunto de datos de prueba***

## Predicciones

***6. Usar cada uno de los 3 modelos para hacer predicciones sobre la 6ª imagen de test (test_images[5]).***

## Gráficas


***7. Utilice el siguiente código para graficar cómo de bien o de mal se comporta el modelo para cada uno de los 3 casos con las 14 primeras imágenes del conjunto de test. Reflexione y comente las diferencias que observa.***

## Comparativa con la Práctica 1


***8. Hacer comparativa con los resultados que obtuvisteis en la práctica 1, con las
capas Dense.***

##Mejoras del modelo

###1ª Mejora del modelo

***9. Ver los conceptos de 'batch_normalization' y 'dropout'; y ver si se podría mejorar
el modelo con ello.
https://keras.io/api/layers/normalization_layers/batch_normalization/
https://keras.io/api/layers/regularization_layers/dropout/***

###2ª Mejora del modelo

***9. Ver los conceptos de 'callbacks' (Decaimiento del ratio de aprendizaje); y ver si se
podría mejorar el modelo con ello.
https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Learning
RateScheduler y https://keras.io/api/callbacks/***

# CONCLUSIONES

# REFLEXIONES

# REFERENCIAS BIBLIOGRÁFICAS

- Apuntes recogidos de GitHub y GoogleColab, a raíz de los contenidos estudiados en el aula